In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
#Defining functions.
import requests

#Extracts a pandas dataframe from json format of API.
def get_data(url):
    request = requests.get(url)
    text = request.json()['Data']
    data = pd.DataFrame(text)
    return data

#Downloading complete history from cryptocompare.
def history(time, symbol):
    if time not in ['minute', 'hour', 'day']:
        raise ValueError('Invalid time interval.')

    url = 'https://min-api.cryptocompare.com/data/histo' + time + '?fsym=' + symbol + '&tsym=USD&limit=2000'
    data = get_data(url)  
    while data['open'][0] != 0:
        last = data['time'][0]
        old_data = get_data(url + '&toTs=' + str(last))
        data = old_data.append(data)
        data.reset_index(inplace=True, drop=True)
        
    data = data.drop(data[data['open'] == 0].index)  
    return data

#Get array of prices.
def price(time, symbol, limit=None):
    hist = history(time, symbol)
    full = np.array(hist['open'].values)
    
    if limit:
        exclude = len(full) - limit
    else:
        exclude = None
        
    return full[exclude:]

In [3]:
array = price('hour', 'BTC', 2000)

In [4]:
#ARIMA 
from statsmodels.tsa.arima_model import ARIMA
from sklearn import metrics 

model_arima = ARIMA(array, (5, 1, 1))
model_arima_fit = model_arima.fit(disp=0)
forecast = model_arima_fit.forecast()
pred_arima, ci_95 = str(forecast[0][0]), str(1.96*forecast[1][0])

print("The 95% confidence interval for the next price is " + pred_arima + " ± " + ci_95)

The 95% confidence interval for the next price is 3732.7694697871166 ± 57.05481036561356


In [5]:
#RNN
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Flatten, Dense

scaler = MinMaxScaler(feature_range = (-1, 1))
series_scaled = scaler.fit_transform(array.reshape(-1, 1))

#Enter number of previous values to use and batch size. 
n = 4
b = 8
time_series = TimeseriesGenerator(series_scaled, series_scaled, length=n, batch_size=b)

model_rnn = Sequential()
model_rnn.add(LSTM(256, input_shape=(n, 1)))
model_rnn.add(Dense(1))
model_rnn.compile(optimizer='sgd', loss='mse')
model_rnn.fit_generator(time_series, steps_per_epoch=len(series_scaled)//b, epochs=10, verbose=0)

last = series_scaled[-n:].reshape(1, n, 1)
pred = model_rnn.predict(last)
value = scaler.inverse_transform(pred)[0][0]

print("The neural network predicts the next price to be " + str(value))

The neural network predicts the next price to be 3742.0032
